## Импорт необходимых библиотек

In [6]:
import os
import re
import pandas as pd

from tqdm.auto import tqdm
from wordfreq import top_n_list, zipf_frequency

## Создание частотного словаря

Для решения задачи, нам необходимо создать словарь, где для каждого слова будет указана его частота встречаемости в языке
Алгоритм будет опираться на эти частоты, чтобы выбрать наиболее вероятное разбиение строки на слова

Берем по 800,000 самых частых слов из русского и английского языков и создаем на этой основе словарь

In [7]:
print("Загрузим частоты для русского и английского языков")
words_ru = top_n_list('ru', 800_000)
words_en = top_n_list('en', 800_000)

# Создаждим словарь частот
freq = {w: zipf_frequency(w, 'ru') for w in words_ru}
for w in words_en:
    if w not in freq:
        freq[w] = zipf_frequency(w, 'en')

print(f"Создан словарь. Всего слов: {len(freq)}")

# Сохраняем словарь
with open('ru_en_dict.tsv', 'w', encoding='utf-8') as f:
    for w, z in sorted(freq.items(), key=lambda x: -x[1]):
        f.write(f"{w}\t{z:.3f}\n")

print("Файл 'ru_en_dict.tsv' создан")

Загрузим частоты для русского и английского языков
Создан словарь. Всего слов: 995563
Файл 'ru_en_dict.tsv' создан


## Вспомогательные функции

Функция загрузки созданного нами словаря 

In [ ]:
def load_word_freq_dict(dict_path='ru_en_dict.tsv'):
    print(f"Загрузка словаря частот из файла '{dict_path}'")

    if not os.path.exists(dict_path):
        print(f"Ошибка. Файл словаря '{dict_path}' не найден"); return None
    
    freq_dict = {}
    one_letter_whitelist = {'в', 'с', 'и', 'к', 'у', 'а', 'a', 'i'}

    with open(dict_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                word, freq = parts
                if len(word) > 1 or word in one_letter_whitelist:
                    freq_dict[word] = float(freq)
    
    for word in one_letter_whitelist:
        if word not in freq_dict: freq_dict[word] = 5.0
        
    print(f"Словарь готов. Уникальных слов: {len(freq_dict)}")
    return freq_dict

### Опишем функции алгоритма сегментации

Функция segment_text:
Для каждого префикса строки она находит самое вероятное разбиение на слова, основываясь на частотах из словаря.
Качество разбиения оценивается как сумма частот слов, умноженных на бонус за длину слова. Бонус помогает алгоритму предпочитать более длинные и осмысленные слова вместо дробления на короткие куски.
Если слово не найдено в словаре, ему присваивается большой отрицательный штраф, чтобы алгоритм избегал таких вариантов.

In [8]:
def segment_text(text, freq_dict):
    n = len(text)
    memo = {0: (0.0, 0)}
    unknown_word_freq = -20.0

    for i in range(1, n + 1):
        best_score = -float('inf'); best_len = 0
        max_word_len = min(i, 25) 

        for k in range(1, max_word_len + 1):
            start_pos = i - k; word = text[start_pos:i]
            word_len_bonus = len(word) ** 2
            base_freq = freq_dict.get(word, unknown_word_freq)
            current_word_score = base_freq * word_len_bonus
            prev_score, _ = memo.get(start_pos, (-float('inf'), 0))
            current_total_score = prev_score + current_word_score

            if current_total_score > best_score:
                best_score = current_total_score; best_len = k

        memo[i] = (best_score, best_len)

    words = []
    i = n

    while i > 0:
        _, k = memo[i]
        if k == 0: break
        words.append(text[i-k:i]); i -= k

    return words[::-1]


Функция segmenter:
Будет обрабатывать строки, которые содержат содержат русские буквы, латиницу, цифры и знаки препинания.
1. Текст будет разбиваться на островки кириллицы, латиницы, цифр и прочих символов
2. Каждый "островок" кириллицы или латиницы будет обрабатываться через функцию segment_text
3. Цифры и символы добавляем как есть
4. В конце сделаем склейку, чтобы знаки препинания присоединялись к предыдущему слову

In [11]:
def segmenter(text, freq_dict):
    # Найдем блоки кириллицы, латиницы, цифр и тд
    pattern = r'([а-яё]+|[a-z]+|[0-9]+|[^а-яёa-z0-9\s]+)'
    parts = re.findall(pattern, text.lower())
    
    raw_tokens = []
    for part in parts:
        if not part: continue
        # Если часть - это кириллица ИЛИ латиница, сегментируем ее
        if re.fullmatch(r'[а-яё]+', part) or re.fullmatch(r'[a-z]+', part):
            raw_tokens.extend(segment_text(part, freq_dict))
        # Иначе (цифры, символы) - добавим как есть
        else:
            raw_tokens.append(part)

    if not raw_tokens:
        return []

    # Приклеим пунктуацию
    final_tokens = [raw_tokens[0]]
    punctuation_pattern = r'^[.,!?;:)\]}]+$'
    for token in raw_tokens[1:]:
        if re.fullmatch(punctuation_pattern, token):
            if final_tokens:
                final_tokens[-1] += token
        else:
            final_tokens.append(token)
            
    return final_tokens


## Обработка и создание submission

Вызовем все описанные выше функции по очереди

1. Загрузим частотный словрь
2. Для каждой строки из task_data.txt вызывает segmenter и получает список слов.
3. На основе списка слов вычисляет позиции, где должны стоять пробелы.
4. Собирает все результаты в список submission

In [ ]:
def main():
    word_freq_dict = load_word_freq_dict('ru_en_dict.tsv')
    if word_freq_dict is None: return

    input_file = 'task_data.txt'

    ids, texts = [], []
    with open(input_file, 'r', encoding='utf-8') as f:
        next(f);
        for line in f:
            line = line.strip()
            if not line: continue
            parts = line.split(',', 1)
            ids.append(parts[0]); texts.append(parts[1] if len(parts) > 1 else "")
    data = pd.DataFrame({'id': ids, 'text_no_spaces': texts})

    all_results_for_submission = []

    for _, row in tqdm(data.iterrows(), total=data.shape[0]):
        text_id = int(row['id'])
        text_no_spaces = str(row['text_no_spaces'])
        
        segmented_words = segmenter(text_no_spaces, word_freq_dict)
        
        predicted_positions = []
        current_pos = 0
        if len(segmented_words) > 1:
            for i in range(len(segmented_words) - 1):
                current_pos += len(segmented_words[i])
                predicted_positions.append(current_pos)
        
        all_results_for_submission.append({'id': text_id, 'predicted_positions': str(predicted_positions)})
        
    submission_df = pd.DataFrame(all_results_for_submission)
    submission_df.to_csv('submission.csv', index=False)
    print("Файл 'submission.csv' создан.")

In [13]:
main()

Загрузка словаря частот из файла 'ru_en_dict.tsv'...
Словарь готов. Уникальных слов: 994310
Чтение через pandas не удалось, пробуем вручную...


100%|██████████| 1005/1005 [00:00<00:00, 6429.36it/s]

Файл 'submission.csv' создан.
